In [1]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository
from IPython.display import VimeoVideo

In [ ]:
#connecting to the database
connection =sqlite3.connect(settings.db_name,check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

In [ ]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_aapl = GarchModel(ticker="AAPL", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_aapl.ticker == "AAPL"
assert gm_aapl.repo == repo
assert not gm_aapl.use_new_data
assert gm_aapl.model_directory == settings.model_directory

In [ ]:
settings.model_directory


In [ ]:
# Instantiate `GarchModel`, use new data
model_tsla = GarchModel(ticker="TSLA", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_tsla, "data")

# Wrangle data
model_tsla.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_tsla, "data")

# Is the `data` a Series?
assert isinstance(model_tsla.data, pd.Series)

# Is Series correct shape?
assert model_tsla.data.shape == (1000,)

model_tsla.data.head()

In [ ]:
# Instantiate `GarchModel`, use old data
model_tsla = GarchModel(ticker="TSLA", repo=repo, use_new_data=False)

# Wrangle data
model_tsla.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_tsla.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr (model_tsla, "model")

# Is model correct data type?
assert isinstance(model_tsla.model, ARCHModelResult)

# Does model have correct parameters?
assert model_tsla.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_tsla.model.summary()

In [ ]:
# Generate prediction from `model_tsla`
prediction =model_tsla.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

In [ ]:
# just stater code trying to craete the path where to save our model
model_directory=settings.model_directory
ticker="TSLA"
timestamp=pd.Timestamp.now().isoformat()
filepath=os.path.join(model_directory,f"{timestamp}_{ticker}.pkl")
filepath

In [ ]:
# Save `model_shop` model, assign filename
filename = model_tsla.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_tsla.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

In [ ]:
#load function for our model
#Add try and except error handling blocks
def load(ticker):
    """Load latest model from model directory.
    
    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.
    
    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")
    
    # Try to find path of latest model
    try:
        model_path = sorted(glob(pattern))[-1]
    # Handle possible `IndexError`
    except IndexError:
        raise Exception(f"No model trained for '{ticker}'.")
    
    # Load model
    model = joblib.load(model_path)
    
    # Return model
    return model

In [ ]:
# Assign load output to `model`
model_tsla= load(ticker="TSLA")
#Testing the function 

# Does function return an `ARCHModelResult`
assert isinstance(model_tsla, ARCHModelResult)

# Check model parameters
model_tsla.summary()

In [ ]:
model_tsla= GarchModel(ticker="TSLA", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_tsla, "model")

# Load model
model_tsla.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_tsla, "model")

model_tsla.model.summary()

In [ ]:
#code for launching the sever in the terminal for the fast API app to run in the uvicorn 
#uvicorn main:app --reload --workers 1 --host localhost --port 8008

In [ ]:
#example of a path to check if our server is working
url ="http://localhost:8008/hello"
response =requests.get(url=url)

print("response code:", response.status_code)
response.json()

In [ ]:
#Checking if the fit in and fit out data classes are working
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker="TSLA",
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1
)
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
    ticker="TSLA",
    use_new_data=True,
    n_observations=2000,
    p=1,
    q=1,
    success=True,
    message="Model is ready to rock!!!"
)
print(fo)

In [ ]:
#model helper  function assert statements
from main import build_model

# Instantiate `GarchModel` with function
model_tsla = build_model(ticker="TSLA", use_new_data=False)

# Is `SQLRepository` attached to `model_shop`?
assert isinstance(model_tsla.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_tsla.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_tsla.ticker == "TSLA"

# Is `use_new_data` attribute correct?
assert not model_tsla.use_new_data

model_tsla

In [ ]:
#checking if the fit_model path is working sucessfully
# URL of `/fit` path
url ="http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker":"TSLA",
    "use_new_data":False,
    "n_observations":2000,
    "p":1,
    "q":1
    
}
# Response of post request
response =requests.post(url=url,json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

In [ ]:
#checking if the AIC and BIC attributes are in the model
# Import `build_model` function
from main import build_model

# Build model using new `MTNOY` data
model = build_model(ticker="TSLA", use_new_data=True)

# Wrangle `TSLA` returns
model.wrangle_data(n_observations=2500)

# Fit GARCH(1,1) model to data
model.fit(p=1, q=1)

# Does model have AIC and BIC attributes?
assert hasattr(model, "aic")
assert hasattr(model, "bic")

In [ ]:
#trying to crush the system by passing wrong use_new_data type instead of false i pass in hello
# URL of `/fit` path
url ="http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker":"TSLA",
    "use_new_data":"hello",
    "n_observations":2000,
    "p":1,
    "q":1
    
}
# Response of post request
response =requests.post(url=url,json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

In [ ]:
#try another ticker symbol for another stock nividia
# URL of `/fit` path
url ="http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker":"NVDA",
    "use_new_data":True,
    "n_observations":2000,
    "p":1,
    "q":1
    
}
# Response of post request
response =requests.post(url=url,json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

In [ ]:
#testing the predict in and predict out classes of the predict path
from main import PredictIn, PredictOut

pi = PredictIn(ticker="TSLA", n_days=5)
print(pi)

po = PredictOut(
    ticker="TSLA", n_days=5, success=True, forecast={}, message="success"
)
print(po)

In [ ]:
#Post request to the predict path to assert it is working
# URL of `/predict` path
url = "http://localhost:8008/predict"
# Data to send to path
json = {"ticker":"TSLA","n_days":5
}
# Response of post request
response = requests.post(url=url,json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

In [ ]:
# Import `FitIn` class and `fit_model` function
from main import FitIn, fit_model

# Instantiate `FitIn` object
request = FitIn(ticker="TSLA", use_new_data=False, n_observations=2500, p=1, q=1)

# Build model and fit to data, following parameters in `request`
fit_out = fit_model(request=request)

# Inspect `fit_out`
fit_out